# Notes

# Links
(Extentions in Webdriver)[https://www.reddit.com/r/learnpython/comments/4zzn69/how_do_i_get_adblockplus_to_work_with_selenium/]

In [282]:
# Imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import time

In [283]:
# Settings
root_url = r'https://www.wowhead.com/wotlk'
output_dir = r'output'

min_itemlvl = 0
max_itemlvl = 284

In [ ]:
# Setup
uBlock_Origin_Lite_extention_path = Path("uBlock_Origin_Lite/2025.612.1428_0").absolute()

Path(output_dir).mkdir(parents=True, exist_ok=True)

driver_options = webdriver.chrome.options.Options()
driver_options.page_load_strategy = 'normal'
driver_options.add_argument(r'--headless')
driver = webdriver.Chrome(options=driver_options)

In [285]:
# Setup item scraping
items_url = root_url + "/items"
driver.get(items_url)

In [286]:
# Scrape item quality and build itemlist links
def process_item_quality_elem(elem):
    _id = int(elem.get_attribute("value"))
    _name = elem.text
    _color = elem.value_of_css_property("color")
    _color = _color[_color.index('(') + 1:]
    _color = _color[:_color.index(')')]
    _color_split = _color.split(', ')
    _color = '#' + "{:02x}".format(int(_color_split[0])) + "{:02x}".format(
        int(_color_split[1])) + "{:02x}".format(int(_color_split[2]))
    return {"id": _id, "name": _name, "color": _color}


item_quality_elems = driver.find_elements(
    By.CSS_SELECTOR, '#filter-facet-quality > option')
item_qualities = pd.DataFrame.from_records(
    [process_item_quality_elem(e) for e in item_quality_elems],
    index='id'
)

item_qualities.to_csv(output_dir + "/item_qualities.csv", sep=';')

itemlist_urls = []
for quality in item_qualities.itertuples():
    for itemlvl in range(min_itemlvl, max_itemlvl):
        itemlist_url = "{u}/min-level:{l:n}/max-level:{l:n}/quality:{q:n}".format(
            u=items_url,
            l=itemlvl,
            q=quality.Index
        )
        itemlist_urls.append(itemlist_url)

In [287]:
# Scrape itemslists for item urls
def process_itemlist(url):
    click_next_btn_js = r'next_btn = Array.from(document.querySelectorAll("#tab-items > div.listview-band-top > div.listview-nav > a")).find(x => x.textContent.toLowerCase().startsWith("next")); if(next_btn !== undefined) next_btn.click()'
    get_item_links_js = r'return Array.from(document.querySelectorAll("#tab-items > div.listview-scroller-horizontal > div > table > tbody > tr > td:nth-child(2) > div > a")).map(x => x.href)'
    driver.get(url)
    frames = []
    pre_url = ""
    while driver.current_url != pre_url:
        item_links = driver.execute_script(get_item_links_js)
        frames.append(pd.DataFrame(item_links))
        pre_url = driver.current_url
        driver.execute_script(click_next_btn_js)

    return pd.concat(frames)


item_url_frames = []
for u in tqdm(itemlist_urls):
    item_url_frames.append(process_itemlist(u))

item_urls = pd.concat(item_url_frames).drop_duplicates().rename(
    columns={0: "url"})
item_urls.sort_values(by=['url'], inplace=True, ignore_index=True)
item_urls.to_csv(output_dir + "/item_urls.csv",
                 sep=';', index=None, header=None)
item_urls

  1%|          | 19/2272 [00:25<51:14,  1.36s/it] 


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=137.0.7151.104)
Stacktrace:
	GetHandleVerifier [0x0x7ff7eae0fe75+79173]
	GetHandleVerifier [0x0x7ff7eae0fed0+79264]
	(No symbol) [0x0x7ff7eabc9e5a]
	(No symbol) [0x0x7ff7eaba20f1]
	(No symbol) [0x0x7ff7eac5017e]
	(No symbol) [0x0x7ff7eac70922]
	(No symbol) [0x0x7ff7eac48743]
	(No symbol) [0x0x7ff7eac114c1]
	(No symbol) [0x0x7ff7eac12253]
	GetHandleVerifier [0x0x7ff7eb0da2ad+3004797]
	GetHandleVerifier [0x0x7ff7eb0d46fd+2981325]
	GetHandleVerifier [0x0x7ff7eb0f3350+3107360]
	GetHandleVerifier [0x0x7ff7eae2a9fe+188622]
	GetHandleVerifier [0x0x7ff7eae3228f+219487]
	GetHandleVerifier [0x0x7ff7eae18dc4+115860]
	GetHandleVerifier [0x0x7ff7eae18f79+116297]
	GetHandleVerifier [0x0x7ff7eadff528+11256]
	BaseThreadInitThunk [0x0x7ffbce637374+20]
	RtlUserThreadStart [0x0x7ffbce7dcc91+33]


In [ ]:
# shutdown
driver.close()